In [1]:
# make deterministic
from minGPT.mingpt.utils import set_seed
set_seed(42)

In [2]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [158]:
from torch.utils.data import Dataset

class AdditionDataset(Dataset):
    def __init__(self, ndigit, split):
        self.split = split # train/test
        self.ndigit = ndigit
        self.vocab_size = 10 # 10 possible digits 0..9
        # +1 due to potential carry overflow, but then -1 because very last digit doesn't plug back
        self.block_size = ndigit + ndigit + ndigit + 1 - 1
        
        # split up all addition problems into either training data or test data
        num = (10**self.ndigit)**2 # total number of possible combinations
        r = np.random.RandomState(1337) # make deterministic
        perm = r.permutation(num)
        num_test = min(int(num*0.2), 1000) # 20% of the whole dataset, or only up to 1000
        self.ixes = perm[:num_test] if split == 'test' else perm[num_test:]

    def __len__(self):
        return self.ixes.size

    def __getitem__(self, idx):
        # given a problem index idx, first recover the associated a + b
        idx = self.ixes[idx]
        nd = 10**self.ndigit
        a = idx // nd
        b = idx %  nd
        c = a + b
        render = f'%0{self.ndigit}d%0{self.ndigit}d%0{self.ndigit+1}d' % (a,b,c) # e.g. 03+25=28 becomes "0325028" 
        dix = [int(s) for s in render] # convert each character to its token index
        # x will be input to GPT and y will be the associated expected outputs
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long) # predict the next token in the sequence
        y[:self.ndigit*2-1] = -100 # we will only train in the output locations. -100 will mask loss to zero
        return x, y


In [212]:
# create a dataset for e.g. 2-digit addition
ndigit = 1
train_dataset = FactoringDataset(ndigit=ndigit, base=64, split='train')
test_dataset = FactoringDataset(ndigit=ndigit, base=64, split='test')

In [213]:
train_dataset[0]

ValueError: Bases greater than 36 not handled in base_repr.

In [209]:
from minGPT.mingpt.model import GPT, GPTConfig, GPT1Config

# initialize a baby GPT model
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size, 
                  n_layer=8, n_head=8, n_embd=64)
model = GPT(mconf)


11/12/2020 22:22:56 - INFO - minGPT.mingpt.model -   number of parameters: 4.024960e+05


In [211]:
from importlib import reload
import json
import os
import minGPT.mingpt.trainer
reload(minGPT.mingpt.trainer)
from minGPT.mingpt.trainer import Trainer, TrainerConfig
# with 1024*4 batch size 1 layer 1 head n_embed=128 stalled out at 0.05 test and 0.25 train
# initialize a trainer instance and kick off training
# 5:11 for epoch at 16
# 01:04 for epoch at 512
# 01:02 for epoch at 2048
# 1:20 for epoch at 2048*4



saveString = f'layer{mconf.n_layer}head{mconf.n_head}emb{mconf.n_embd}{str(type(train_dataset).__name__)}base{train_dataset.base}digit{train_dataset.ndigit}'
lossesPath = saveString + "losses.json"
tconf = TrainerConfig(max_epochs=10000, batch_size=1024, learning_rate=3e-4,
                      lr_decay=True, warmup_tokens=0, final_tokens=50*len(train_dataset)*(ndigit+1),
                      num_workers=0, ckpt_path=saveString, losses_path=lossesPath)
trainer = Trainer(model, train_dataset, test_dataset, tconf, useCuda=True)
while True:
    from IPython.display import clear_output
    def doCallback():
        give_exam(train_dataset, batch_size=1024, max_batches=10)
        give_exam(test_dataset, batch_size=1024, max_batches=10)
    trainer.train(doCallback)
    clear_output(wait=True)
    tconf.warmup_tokens = 0
    

11/12/2020 22:23:19 - INFO - minGPT.mingpt.trainer -   loading from checkpoint layer8head8emb64FactoringDatasetbase16digit2
11/12/2020 22:23:19 - INFO - minGPT.mingpt.trainer -   loading losses from checkpoint layer8head8emb64FactoringDatasetbase16digit2losses.json
epoch 1 iter 2: train loss 2.64284. lr 2.994739e-04: 100%|██████████| 3/3 [00:00<00:00,  6.25it/s]
11/12/2020 22:23:20 - INFO - minGPT.mingpt.trainer -   test loss: 2.608861
11/12/2020 22:23:20 - INFO - minGPT.mingpt.trainer -   saving layer8head8emb64FactoringDatasetbase16digit2
  0%|          | 0/3 [00:00<?, ?it/s]

final score: 2/2333 = 0.09% correct
final score: 0/583 = 0.00% correct


epoch 2 iter 2: train loss 2.58420. lr 2.978994e-04: 100%|██████████| 3/3 [00:00<00:00,  6.41it/s]
11/12/2020 22:23:21 - INFO - minGPT.mingpt.trainer -   test loss: 2.547491
11/12/2020 22:23:21 - INFO - minGPT.mingpt.trainer -   saving layer8head8emb64FactoringDatasetbase16digit2
epoch 3 iter 2: train loss 2.52601. lr 2.952875e-04: 100%|██████████| 3/3 [00:00<00:00,  6.50it/s]
11/12/2020 22:23:21 - INFO - minGPT.mingpt.trainer -   test loss: 2.502897
11/12/2020 22:23:21 - INFO - minGPT.mingpt.trainer -   saving layer8head8emb64FactoringDatasetbase16digit2
epoch 4 iter 2: train loss 2.50215. lr 2.916565e-04: 100%|██████████| 3/3 [00:00<00:00,  6.53it/s]
11/12/2020 22:23:22 - INFO - minGPT.mingpt.trainer -   test loss: 2.463198
11/12/2020 22:23:22 - INFO - minGPT.mingpt.trainer -   saving layer8head8emb64FactoringDatasetbase16digit2
epoch 5 iter 2: train loss 2.45072. lr 2.870318e-04: 100%|██████████| 3/3 [00:00<00:00,  6.48it/s]
11/12/2020 22:23:22 - INFO - minGPT.mingpt.trainer -   tes

final score: 18/2333 = 0.77% correct
final score: 3/583 = 0.51% correct


epoch 22 iter 2: train loss 2.10692. lr 1.096620e-04: 100%|██████████| 3/3 [00:00<00:00,  6.38it/s]
11/12/2020 22:23:32 - INFO - minGPT.mingpt.trainer -   test loss: 2.099664
11/12/2020 22:23:32 - INFO - minGPT.mingpt.trainer -   saving layer8head8emb64FactoringDatasetbase16digit2
epoch 23 iter 2: train loss 2.10657. lr 9.771419e-05: 100%|██████████| 3/3 [00:00<00:00,  6.37it/s]
11/12/2020 22:23:33 - INFO - minGPT.mingpt.trainer -   test loss: 2.086666
11/12/2020 22:23:33 - INFO - minGPT.mingpt.trainer -   saving layer8head8emb64FactoringDatasetbase16digit2
epoch 24 iter 2: train loss 2.10408. lr 8.613311e-05: 100%|██████████| 3/3 [00:00<00:00,  6.17it/s]
11/12/2020 22:23:33 - INFO - minGPT.mingpt.trainer -   test loss: 2.082760
11/12/2020 22:23:33 - INFO - minGPT.mingpt.trainer -   saving layer8head8emb64FactoringDatasetbase16digit2
epoch 25 iter 2: train loss 2.10511. lr 7.500000e-05: 100%|██████████| 3/3 [00:00<00:00,  6.14it/s]
11/12/2020 22:23:34 - INFO - minGPT.mingpt.trainer -  

...

final score: 234/2333 = 10.03% correct
final score: 12/583 = 2.06% correct


epoch 442 iter 2: train loss 1.10624. lr 2.675540e-04: 100%|██████████| 3/3 [00:00<00:00,  5.53it/s]
11/12/2020 22:27:24 - INFO - minGPT.mingpt.trainer -   test loss: 1.126918
epoch 443 iter 2: train loss 1.11807. lr 2.749382e-04: 100%|██████████| 3/3 [00:00<00:00,  5.49it/s]
11/12/2020 22:27:24 - INFO - minGPT.mingpt.trainer -   test loss: 1.129481
epoch 444 iter 2: train loss 1.09457. lr 2.814460e-04: 100%|██████████| 3/3 [00:00<00:00,  5.64it/s]
11/12/2020 22:27:25 - INFO - minGPT.mingpt.trainer -   test loss: 1.134041
epoch 445 iter 2: train loss 1.08715. lr 2.870318e-04: 100%|██████████| 3/3 [00:00<00:00,  6.57it/s]
11/12/2020 22:27:25 - INFO - minGPT.mingpt.trainer -   test loss: 1.138533
epoch 446 iter 2: train loss 1.13210. lr 2.916565e-04: 100%|██████████| 3/3 [00:00<00:00,  6.44it/s]
11/12/2020 22:27:26 - INFO - minGPT.mingpt.trainer -   test loss: 1.147942
epoch 447 iter 2: train loss 1.10229. lr 2.952875e-04: 100%|██████████| 3/3 [00:00<00:00,  6.41it/s]
11/12/2020 22:27:26

final score: 311/2333 = 13.33% correct
final score: 22/583 = 3.77% correct


epoch 462 iter 2: train loss 1.05749. lr 2.303740e-04: 100%|██████████| 3/3 [00:00<00:00,  6.38it/s]
11/12/2020 22:27:35 - INFO - minGPT.mingpt.trainer -   test loss: 1.114174
11/12/2020 22:27:35 - INFO - minGPT.mingpt.trainer -   saving layer8head8emb64FactoringDatasetbase16digit2
epoch 463 iter 2: train loss 1.07924. lr 2.194944e-04: 100%|██████████| 3/3 [00:00<00:00,  6.39it/s]
11/12/2020 22:27:35 - INFO - minGPT.mingpt.trainer -   test loss: 1.124339
epoch 464 iter 2: train loss 1.06731. lr 2.081273e-04: 100%|██████████| 3/3 [00:00<00:00,  6.41it/s]
11/12/2020 22:27:36 - INFO - minGPT.mingpt.trainer -   test loss: 1.105644
11/12/2020 22:27:36 - INFO - minGPT.mingpt.trainer -   saving layer8head8emb64FactoringDatasetbase16digit2
epoch 465 iter 2: train loss 1.05559. lr 1.963525e-04: 100%|██████████| 3/3 [00:00<00:00,  6.39it/s]
11/12/2020 22:27:36 - INFO - minGPT.mingpt.trainer -   test loss: 1.102253
11/12/2020 22:27:36 - INFO - minGPT.mingpt.trainer -   saving layer8head8emb64Fact

KeyboardInterrupt: 

In [182]:
# now let's give the trained model an addition exam
from torch.utils.data.dataloader import DataLoader
from minGPT.mingpt.utils import sample

def give_exam(dataset, batch_size=32, max_batches=-1, showFailures=False, failureCallback=None):
    
    results = []
    loader = DataLoader(dataset, batch_size=batch_size)
    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        y = y.to(trainer.device)
        numMask = (y[0]==-100).int().sum() # we mask away the inputs, so sum
        numOutput = y[0].shape[0]-numMask
        d1d2 = x[:, :numMask+1] # +1 because y is shifted one so we actually mask two things
        d1d2d3 = sample(model, d1d2, numOutput).to(trainer.device)
        desiredOutput = y[:,numMask:]
        modelOutput = d1d2d3[:,numMask+1:]
        correct = (desiredOutput == modelOutput).cpu().all(axis=1) 
        for i in range(x.size(0)):
            results.append(int(correct[i]))
            judge = 'YEP!!!' if correct[i] else 'NOPE'
            if not correct[i] and showFailures:
                print(f"GPT claims that {str(x[i].cpu().numpy())} gives {str(d1d2d3[i][1:].cpu().numpy())} actual is {str(y[i].cpu().numpy())}")
            if not correct[i] and failureCallback is not None:
                failureCallback(x[i].cpu().numpy(), y[i].cpu().numpy(), d1d2d3[i][1:].cpu().numpy())
        
        if max_batches >= 0 and b+1 >= max_batches:
            break

    print("final score: %d/%d = %.2f%% correct" % (np.sum(results), len(results), 100*np.mean(results)))

In [191]:
import customDatasets
from importlib import reload
reload(customDatasets)

def toNum(base, arr):
    a = arr
    if type(arr) == torch.tensor:
        a = arr.cpu().numpy()
    val = 0
    for i, x in enumerate(a[::-1]):
        val += x*(base**i)
    return val
        
def multArrays(base, a, b):
    aVal = toNum(base, a)
    bVal = toNum(base, b)
    return aVal*bVal

base = 32
print(datasets.prime(toNum(base, [24, 29])))
print(convertArrayToNum(base, [9,17,31,25]), multArrays(base, [12, 13], [24, 29]))

True
313337 316409


In [202]:
global numPrimes
global numNotPrimes
numPrimes = 0
numNotPrimes = 0
def failureFactor(x, y, networkOutput):
    global numPrimes
    global numNotPrimes
    nDigits = (len(x)+1)//4
    productResult = toNum(base, x[:nDigits*2])
    actualA, actualB = toNum(base, y[-nDigits*2:-nDigits]), toNum(base, y[-nDigits:])
    outputA, outputB = toNum(base, networkOutput[-nDigits*2:-nDigits]), toNum(base, networkOutput[-nDigits:])
    print("acutal:", productResult, "=", actualA, "*", actualB, "output", (outputA*outputB), "=", outputA, "*", outputB)
    diff = productResult-outputA*outputB
    if customDatasets.prime(outputA): numPrimes += 1
    else: numNotPrimes += 1
    if customDatasets.prime(outputB): numPrimes += 1
    else: numNotPrimes += 1
    print("difference:", diff, "relative magnitude:", float(diff)/productResult, "isPrime:", outputA, customDatasets.prime(outputA), outputB, customDatasets.prime(outputB))

In [204]:
give_exam(train_dataset, batch_size=1024, max_batches=10, showFailures=True, failureCallback=failureFactor)
print(numPrimes, numNotPrimes)

GPT claims that [0 0 0 6 0 2 0] gives [ 0  0  6  0  2  0 19] actual is [-100 -100 -100    0    2    0    3]
acutal: 6 = 2 * 3 output 38 = 2 * 19
difference: -32 relative magnitude: -5.333333333333333 isPrime: 2 True 19 True
GPT claims that [ 9 17 31 25 13 15 22] gives [17 31 25 12 13 24 29] actual is [-100 -100 -100   13   15   22   23]
acutal: 313337 = 431 * 727 output 316409 = 397 * 797
difference: -3072 relative magnitude: -0.00980414058984416 isPrime: 397 True 797 True
GPT claims that [ 3  2  4  9  9 29  9] gives [ 2  4  9  6  1 16  9] actual is [-100 -100 -100    9   29    9   29]
acutal: 100489 = 317 * 317 output 100553 = 193 * 521
difference: -64 relative magnitude: -0.0006368856292728557 isPrime: 193 True 521 True
GPT claims that [ 2 31 30 21  3  1 31] gives [31 30 21  8 13 11  9] actual is [-100 -100 -100    3    1   31   21]
acutal: 98261 = 97 * 1013 output 97109 = 269 * 361
difference: 1152 relative magnitude: 0.0117238782426395 isPrime: 269 True 361 False
GPT claims that [ 

In [206]:
global numPrimes, numNotPrimes
numPrimes, numNotPrimes = 0,0
give_exam(test_dataset, batch_size=1024, max_batches=10, showFailures=True, failureCallback=failureFactor)
print(numPrimes, numNotPrimes)

GPT claims that [16 16 13 21 19  9 27] gives [16 13 21 20 13 25  9] actual is [-100 -100 -100   19    9   27   13]
acutal: 541109 = 617 * 877 output 528277 = 653 * 809
difference: 12832 relative magnitude: 0.023714260897527116 isPrime: 653 True 809 True
GPT claims that [20  6 29 29 24  5 26] gives [ 6 29 29 22 15 29 19] actual is [-100 -100 -100   24    5   26   25]
acutal: 662461 = 773 * 857 output 680893 = 719 * 947
difference: -18432 relative magnitude: -0.027823524705605313 isPrime: 719 True 947 True
GPT claims that [16  4  4 17 22 23 22] gives [ 4  4 17 19  5 26 25] actual is [-100 -100 -100   22   23   22   23]
acutal: 528529 = 727 * 727 output 525341 = 613 * 857
difference: 3188 relative magnitude: 0.006031835528419444 isPrime: 613 True 857 True
GPT claims that [ 0 10  8 21  0 13 25] gives [10  8 21  2  3  5  7] actual is [-100 -100 -100    0   13   25    9]
acutal: 10517 = 13 * 809 output 11189 = 67 * 167
difference: -672 relative magnitude: -0.06389654844537415 isPrime: 67 Tru

In [33]:
# n_layer=2, n_head=2, n_embd=128 - gets 100%



# factoring:
# base 16, 2 digits:
# 8 layers, 2 heads stalls out at 0.14311, and also seems to be memorizing because test loss goes down but then back up

# 8 layers, 8 heads, embed 64
# epoch 453 iter 3: train loss 0.98656. lr 5.825030e-04:  80%|████████  | 4/5 [00:00<00:00,  7.09it/s]
# epoch 102 iter 0: train loss 1.47119. lr 1.204984e-04:  20%|██        | 1/5 [00:00<00:00,  9.02it/s]